In [1]:
import sys
import os
import numpy as np
import warnings
from tqdm.auto import tqdm
import pickle
import matplotlib.pyplot as plt
import pandas as pd


# Suppress all warnings
warnings.filterwarnings("ignore")

## replace with root project dir
PROJECT_DIR = "/mnt/workspace/__ing/llming/DTC/audio_podcast_qa_assistant"
sys.path.append(PROJECT_DIR)

from utils.utils import (
    initialize_env_variables,
    flatten_list_of_lists,
    sample_from_list,
    read_json_file,
    save_json_file,
    extract_item_by_keys,
    save_to_pickle,
    load_pickle,
    get_json_files_in_dir,
    read_json_file
)

initialize_env_variables()

from utils.questions import (extract_questions,
                             group_questions_by_episode, 
                             openai_process_questions,
                            filter_corrupted_qs,
                            count_question_marks)
from utils.multithread import map_progress
from datasets import load_dataset, Dataset
from utils.ollama import embed_document
from utils.variables import OLLAMA_CLIENT, INDEX_NAME, ES_CLIENT

from transformers import LEDForConditionalGeneration, LEDTokenizer
import torch

from utils.query import (
    elastic_search_text, elastic_search_knn, elastic_search_hybrid_rrf
)
from utils.variables import ES_CLIENT
from utils.query import (
    elastic_search_text, elastic_search_knn, elastic_search_hybrid_rrf
)
from utils.variables import ES_CLIENT
from utils.evaluate import (hit_rate, mrr, retrieve_relevance,
                            retrieve_adjusted_relevance, adjusted_hit_rate,
                            adjusted_mrr)

Initialized environment variables listed in: /mnt/workspace/__ing/llming/DTC/audio_podcast_qa_assistant/.env
Initialized environment variables listed in: /mnt/workspace/__ing/llming/DTC/audio_podcast_qa_assistant/.env
Connected to Elasticsearch


# Evaluating Performance on questions

In [2]:
pickle_file_path = os.path.join(
    PROJECT_DIR, 
    "data/generated_embeddings/vectorized_questions.pkl"
)
vectorized_questions = load_pickle(pickle_file_path)

## MRR & HIT_RATE

In [3]:
search_func_list = [
    elastic_search_text,
    elastic_search_knn,
    elastic_search_hybrid_rrf
]
search_func_keys_list = [
    {"query":"question"},
    {"query_vector":"question_vector"},
    {"query":"question", "query_vector":"question_vector"}    
]
performance = []

for i in range(len(search_func_list)):
    relevance = map_progress(
        f=lambda question_dict: retrieve_relevance(
            question_dict=question_dict,
            search_func=search_func_list[i],
            **search_func_keys_list[i]
        ),
        seq=vectorized_questions,
        max_workers=6,
        verbose=False
    )
    
    performance.append(
        {
            "Search": search_func_list[i].__name__,
            "HR": hit_rate(relevance),
            "MRR": mrr(relevance)
        }
    )
    
pd.DataFrame(performance)

  0%|          | 0/234 [00:00<?, ?it/s]

  0%|          | 0/234 [00:00<?, ?it/s]

  0%|          | 0/234 [00:00<?, ?it/s]

,Search,HR,MRR
0,elastic_search_text,0.641026,0.503775
1,elastic_search_knn,0.705128,0.586111
2,elastic_search_hybrid_rrf,0.786325,0.671795


## Adjusted MRR & HIT_RATE

In [4]:
search_func_list = [
    elastic_search_text,
    elastic_search_knn,
    elastic_search_hybrid_rrf
]
search_func_keys_list = [
    {"query":"question"},
    {"query_vector":"question_vector"},
    {"query":"question", "query_vector":"question_vector"}    
]
performance = []

for i in range(len(search_func_list)):
    relevance = map_progress(
        f=lambda question_dict: retrieve_adjusted_relevance(
            question_dict=question_dict,
            search_func=search_func_list[i],
            **search_func_keys_list[i]
        ),
        seq=vectorized_questions,
        max_workers=6,
        verbose=False
    )
    
    performance.append(
        {
            "Search": search_func_list[i].__name__,
            "Adjusted HR": adjusted_hit_rate(relevance),
            "Adjusted MRR": adjusted_mrr(relevance)
        }
    )
    
pd.DataFrame(performance)

  0%|          | 0/234 [00:00<?, ?it/s]

  0%|          | 0/234 [00:00<?, ?it/s]

  0%|          | 0/234 [00:00<?, ?it/s]

,Search,Adjusted HR,Adjusted MRR
0,elastic_search_text,0.643162,0.505912
1,elastic_search_knn,0.722222,0.594017
2,elastic_search_hybrid_rrf,0.799145,0.680520
